## Example of AH3 molecular
### Outline
In this example, we use the codebase to solve the orbital interaction in AH3 molecular. It's molecular structure is given by `automaticTB.examples.structures.AH3` where a carbon atom is used as A with $2s$ and $2p$ orbitals. 

The result is given in Section 4.8 of Albright's book *Orbital Interactions in Chemistry*. To arrive at the result, we first obtain all the symmetry adapted linear combination:

### Finding the symmetry adopted linear combination (SALCs)
Since we have in total 7 starting atomic orbital, we arrive at 7 linear combination in total with representation: $2\times A'$, $1\times A''$, $2\times E'$, furthermore, each subspace of the $E'$ representation are separated into symmetric and anti-symmetric components $A$ and $B$. Interaction is only allowed for linear combinations having the same symmetric properties. 

In [ ]:
from automaticTB.functions import get_namedLCs_from_nncluster
from automaticTB.examples import get_AH3_nncluster

ah3 = get_AH3_nncluster()
named_lcs = get_namedLCs_from_nncluster(ah3)

print("Solving the symmetry adopted linear combinations ...")
for nlc in named_lcs:
    print(nlc.name)
    print(nlc.lc)

print("\nJOB DONE !")

### Plotting the linear combination
To plot the symmetry adopted wavefunction, use the following function

In [ ]:
from automaticTB.functions import plot_molecular_wavefunction_from_linear_combination

print("Plotting Symmetry adopted Linear Combinations")
functions_to_plot = [
    "E_'->A_1 @ 1^th E_'",  # oriented p orbitals
    "E_'->A_1 @ 2^th E_'"   # the molecular orbitals combined from 3 s electron
]
for nlc in named_lcs:
    if str(nlc.name) not in functions_to_plot: continue
    filename = f"{str(nlc.name)}.cube"
    plot_molecular_wavefunction_from_linear_combination(
        nlc.lc, filename
    )
    print(f"wavefunction plotted to {filename}")

### Finding the free interaction
Between the 7 linear combinations, we have in total $7\times 7$ interactions between the states. However, only 5 interaction is allowed. This offers a simplification if to understand the chemical interaction and solving of the energy eigenstates. 

In [ ]:
from automaticTB.functions import get_free_AOpairs_from_nncluster_and_namedLCs
from automaticTB.printing import print_ao_pairs

free_pairs = get_free_AOpairs_from_nncluster_and_namedLCs(
    ah3, named_lcs
)
print_ao_pairs(ah3, free_pairs)

### Eigenvalue of the molecular orbitals
The following code provides the interaction energy between the symmetry adopted molecular orbitals and solves the final eigenenergy and the orbital degeneracy. In the MOlist, `s_px` stands for the combination of the s orbitals that has the same symmetry properties as the `px` orbitals. 

The important result is the degeneracy of the final eigen-states, which agree qualitatively with the molecular orbital diagram for in Albright's book mentioned previously.

In [ ]:
import numpy as np
# Hamiltonian matrix in MO basis:
Hij = np.zeros((7,7))
MOlist = [
        "s", # A_'_1 @ 1^th A_'_1
        "px", # E_'->A_1 @ 1^th E_'
        "py", # E_'->B_2 @ 1^th E_'
        "pz", # A_''_2 @ 1^th A_''_2
        "s_s", # A_'_1 @ 2^th A_'_1
        "s_px", # E_'->A_1 @ 2^th E_'
        "s_py" # E_'->B_2 @ 2^th E_'
    ]

interactions = {
        "s s": -25,
        "px px": -13, "py py": -13, "pz pz": -13,
        "s_s s_s": -14,
        "s_px s_px": -12, "s_py s_py": -12,
        "s s_s": 3.0, "s_s s": 3.0,
        "px s_px": 4.0, "s_px px": 4.0, "py s_py": 4.0, "s_py py": 4.0
    }

for i,mo1 in enumerate(MOlist):
    for j,mo2 in enumerate(MOlist):
        Hij[i,j] = interactions.setdefault(" ".join([mo1,mo2]),0)

print("The Molecular Hamiltonian in MO basis is:")
print(Hij)
w, v = np.linalg.eig(Hij)
print()
print("Solved eigenvalues are:")

start = 0
sortedW = sorted(w)
for sw in sortedW:
    print(f"{sw:>8.2f}")
    